<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft

In [ ]:
import syft as sy
import torch
from torch import nn, optim

# Hook PyTorch
hook = sy.TorchHook(torch)

# Define workers (clients)
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        return self.fc(x)

# Initialize model
model = SimpleModel()

# Dummy data
data_alice = torch.tensor([[0.1, 0.2], [0.3, 0.4]], requires_grad=True).send(alice)
data_bob = torch.tensor([[0.5, 0.6], [0.7, 0.8]], requires_grad=True).send(bob)
target_alice = torch.tensor([[1.0], [0.0]]).send(alice)
target_bob = torch.tensor([[0.0], [1.0]]).send(bob)

# Define optimizer and loss function
opt = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Federated training loop
for epoch in range(10):
    # Training on Alice's data
    model.zero_grad()
    pred_alice = model(data_alice)
    loss_alice = criterion(pred_alice, target_alice)
    loss_alice.backward()
    opt.step()

    # Training on Bob's data
    model.zero_grad()
    pred_bob = model(data_bob)
    loss_bob = criterion(pred_bob, target_bob)
    loss_bob.backward()
    opt.step()

    # Corrected print statement
    print(f'Epoch {epoch}, Loss Alice: {loss_alice.get().item()}, Loss Bob: {loss_bob.get().item()}')